# SMT-WEEX Notebook 1: Data Cleaning & Feature Engineering (v2)
**Project:** smt-weex-2025
**Author:** Jannet Ekka

**Updates in v2:**
- Drop highly correlated features
- Add additional behavioral features
- Better handle class imbalance preparation

## 1. Setup & Authentication

In [ ]:
!pip install -q google-cloud-bigquery google-cloud-storage pandas numpy scikit-learn catboost db-dtypes

In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'smt-weex-2025'
BUCKET = 'smt-weex-2025-models'

!gcloud config set project {PROJECT_ID}

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

bq_client = bigquery.Client(project=PROJECT_ID)
print(f"Connected to project: {PROJECT_ID}")

## 2. Load Data from BigQuery

In [ ]:
query = """
SELECT * FROM `smt-weex-2025.ml_data.whale_features`
"""

df = bq_client.query(query).to_dataframe()
print(f"Loaded {len(df)} rows, {len(df.columns)} columns")
print(f"\nCategory Distribution:")
print(df['category'].value_counts())

## 3. Data Cleaning

In [ ]:
# Remove duplicates
duplicates = df['address'].duplicated().sum()
print(f"Duplicate addresses: {duplicates}")
if duplicates > 0:
    df = df.drop_duplicates(subset=['address'], keep='first')
    print(f"After dedup: {len(df)} rows")

# Fill missing values
missing = df.isnull().sum()
missing = missing[missing > 0]
print(f"\nMissing values: {len(missing)} columns")
if len(missing) > 0:
    print(missing)
    df = df.fillna(0)

In [ ]:
# Handle infinite values
NON_FEATURES = ['address', 'category', 'sub_label']
FEATURE_COLS = [col for col in df.columns if col not in NON_FEATURES]

for col in FEATURE_COLS:
    if df[col].dtype in ['float64', 'int64', 'Float64', 'Int64']:
        inf_count = np.isinf(df[col].astype(float)).sum()
        if inf_count > 0:
            max_val = df[col][~np.isinf(df[col].astype(float))].max()
            df[col] = df[col].replace([np.inf, -np.inf], max_val)
            print(f"{col}: replaced {inf_count} inf values")

## 4. Feature Transformations

In [ ]:
# Log transformations for highly skewed features
SKEWED_COLS = [
    'total_txs', 'outgoing_count', 'incoming_count',
    'outgoing_volume_eth', 'incoming_volume_eth',
    'avg_tx_value_eth', 'max_tx_value_eth', 'std_tx_value_eth',
    'avg_gas_used', 'max_gas_used',
    'unique_counterparties', 'unique_tokens', 'balance_eth'
]

for col in SKEWED_COLS:
    if col in df.columns:
        df[f'{col}_log'] = np.log1p(df[col].clip(lower=0).astype(float))

# Signed log for net_flow_eth
def signed_log(x):
    return np.sign(x) * np.log1p(abs(x))

df['net_flow_eth_signed_log'] = df['net_flow_eth'].astype(float).apply(signed_log)
print(f"Created {len(SKEWED_COLS)} log features + net_flow_eth_signed_log")

In [ ]:
# Derived features

# 1. Activity intensity (tx per day)
df['activity_intensity'] = df['total_txs'].astype(float) / (df['activity_span_days'].astype(float) + 1)

# 2. DeFi engagement score
df['defi_engagement'] = (df['defi_interactions'].astype(float) + df['unique_defi_protocols'].astype(float)) / (df['total_txs'].astype(float) + 1)

# 3. Token diversity normalized
df['token_diversity_norm'] = df['unique_tokens'].astype(float) / (df['erc20_tx_count'].astype(float) + 1)

# 4. Value concentration (max/avg ratio) - indicates large single transactions
df['value_concentration'] = df['max_tx_value_eth'].astype(float) / (df['avg_tx_value_eth'].astype(float) + 0.001)

# 5. Flow imbalance (absolute)
df['flow_imbalance'] = abs(df['net_flow_eth'].astype(float)) / (df['incoming_volume_eth'].astype(float) + df['outgoing_volume_eth'].astype(float) + 0.001)

# 6. Gas efficiency (inverse of avg gas - higher = simpler txs)
df['gas_efficiency'] = 1 / (df['avg_gas_used'].astype(float) + 1)

# 7. NFT activity indicator (binary-ish)
df['is_nft_active'] = (df['nft_ratio'] > 0.01).astype(int)

# 8. Internal tx ratio (indicates contract interactions)
df['internal_tx_heavy'] = (df['internal_ratio'] > 0.1).astype(int)

# 9. Outflow dominance (distribution behavior indicator)
df['outflow_dominant'] = (df['tx_ratio_out_in'] > 1.5).astype(int)

# 10. High frequency trader indicator
df['high_freq_trader'] = (df['tx_per_day'] > 10).astype(int)

print("Created 10 derived features")

## 5. Feature Selection - Drop Highly Correlated

In [ ]:
# Define final ML feature list
# Dropping one from each highly correlated pair:
# - activity_intensity (keep tx_per_day - same thing)
# - unique_defi_protocols (keep defi_interactions - identical)
# - max_tx_value_eth_log (keep avg_tx_value_eth_log - 0.983 corr)
# - incoming_count_log (keep total_txs_log - 0.956 corr)
# - std_time_between_tx_hours (keep avg_time_between_tx_hours - 0.916 corr)

ML_FEATURES = [
    # Original ratio features (low correlation, high signal)
    'erc20_ratio',
    'nft_ratio', 
    'internal_ratio',
    'large_tx_ratio',
    'stablecoin_ratio',
    'tx_ratio_out_in',
    'business_hour_ratio',
    'peak_hour_pct',
    
    # Time features (keep only one from correlated pair)
    'avg_time_between_tx_hours',  # dropped std_time_between_tx_hours
    'tx_per_day',  # dropped activity_intensity (identical)
    
    # Interaction counts
    'defi_interactions',  # dropped unique_defi_protocols (identical)
    'cex_interactions',
    
    # Log-transformed features (dropped highly correlated ones)
    'total_txs_log',  # dropped incoming_count_log (0.956 corr)
    'outgoing_count_log',
    'outgoing_volume_eth_log',
    'incoming_volume_eth_log',
    'avg_tx_value_eth_log',  # dropped max_tx_value_eth_log (0.983 corr)
    'avg_gas_used_log',
    'max_gas_used_log',
    'unique_counterparties_log',
    'unique_tokens_log',
    'balance_eth_log',
    
    # Derived features
    'defi_engagement',
    'token_diversity_norm',
    'value_concentration',
    'flow_imbalance',
    'gas_efficiency',
    'net_flow_eth_signed_log',
    
    # Binary indicators (new)
    'is_nft_active',
    'internal_tx_heavy',
    'outflow_dominant',
    'high_freq_trader'
]

# Verify all features exist
ML_FEATURES = [f for f in ML_FEATURES if f in df.columns]
print(f"Final ML features: {len(ML_FEATURES)}")
print(ML_FEATURES)

In [ ]:
# Verify correlation after dropping
plt.figure(figsize=(14, 12))
corr_matrix = df[ML_FEATURES].corr()
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix (After Dropping High Corr)')
plt.tight_layout()
plt.show()

# Check remaining high correlations
print("\nRemaining correlations > 0.85:")
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.85:
            print(f"{corr_matrix.columns[i]} <-> {corr_matrix.columns[j]}: {corr_matrix.iloc[i, j]:.3f}")

## 6. Class Distribution Analysis

In [ ]:
# Analyze class distribution for imbalance handling
class_counts = df['category'].value_counts()
print("=== Class Distribution ===")
for cat, count in class_counts.items():
    pct = count / len(df) * 100
    print(f"{cat:15s}: {count:4d} ({pct:5.1f}%)")

# Calculate class weights for reference
total = len(df)
n_classes = len(class_counts)
print(f"\n=== Recommended Class Weights (balanced) ===")
for cat, count in class_counts.items():
    weight = total / (n_classes * count)
    print(f"{cat:15s}: {weight:.3f}")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
class_counts.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Class Distribution')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Pie chart
axes[1].pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Class Distribution (%)')

plt.tight_layout()
plt.show()

## 7. Feature Statistics by Category

In [ ]:
# Key distinguishing features by category
key_features = ['tx_per_day', 'defi_interactions', 'internal_ratio', 
                'stablecoin_ratio', 'balance_eth_log', 'unique_counterparties_log']

print("=== Key Features by Category (Mean) ===")
category_stats = df.groupby('category')[key_features].mean().round(3)
print(category_stats)

In [ ]:
# Visualize key feature distributions by category
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feat in enumerate(key_features):
    if feat in df.columns:
        for cat in df['category'].unique():
            data = df[df['category'] == cat][feat].astype(float)
            axes[i].hist(data, bins=30, alpha=0.5, label=cat)
        axes[i].set_title(feat)
        axes[i].legend(fontsize=8)

plt.tight_layout()
plt.show()

## 8. Save Cleaned Data

In [ ]:
# Prepare final dataset
df_clean = df[['address', 'category', 'sub_label'] + ML_FEATURES].copy()

# Final cleanup
df_clean = df_clean.fillna(0)
df_clean = df_clean.replace([np.inf, -np.inf], 0)

print(f"Final dataset shape: {df_clean.shape}")
print(f"Features: {len(ML_FEATURES)}")
print(f"Categories: {df_clean['category'].value_counts().to_dict()}")

In [ ]:
# Save locally and to GCS
df_clean.to_csv('/content/whale_features_cleaned.csv', index=False)
!gsutil cp /content/whale_features_cleaned.csv gs://{BUCKET}/data/whale_features_cleaned.csv
print(f"Saved to gs://{BUCKET}/data/whale_features_cleaned.csv")

In [ ]:
# Save feature config
import json

feature_config = {
    'features': ML_FEATURES,
    'target': 'category',
    'categories': df_clean['category'].unique().tolist(),
    'n_samples': len(df_clean),
    'n_features': len(ML_FEATURES),
    'class_counts': df_clean['category'].value_counts().to_dict(),
    'dropped_correlated': [
        'activity_intensity (same as tx_per_day)',
        'unique_defi_protocols (same as defi_interactions)',
        'max_tx_value_eth_log (0.983 corr with avg)',
        'incoming_count_log (0.956 corr with total_txs)',
        'std_time_between_tx_hours (0.916 corr with avg)'
    ]
}

with open('/content/feature_config.json', 'w') as f:
    json.dump(feature_config, f, indent=2)

!gsutil cp /content/feature_config.json gs://{BUCKET}/data/feature_config.json
print("Feature config saved")
print(json.dumps(feature_config, indent=2))

## Summary

**Data Prepared for Notebook 02:**
- 532 whales, 30 features (reduced from 33)
- Dropped 5 highly correlated features
- Added 4 binary indicator features
- Class imbalance: DeFi_Trader 35% to Institutional 6%

**Expected Model Performance (from research):**
- 70-80% balanced accuracy
- 65-75% macro F1
- CatBoost with `auto_class_weights='Balanced'` recommended

**Next:** Run Notebook 02 for model training